In [4]:
import pandas as pd
# df = pd.read_csv('filename.csv')
# df.shape

In [5]:
df.describe()

NameError: name 'df' is not defined

In [6]:
X = df.drop(columns=[genre]) #input set
y = df[genre] #predictions/output set

NameError: name 'df' is not defined

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [9]:
model = DecisionTreeClassifier()
model.fit(X,y)
predictions = model.predict([[21,1],[22,0]])
predictions

NameError: name 'X' is not defined